<a href="https://colab.research.google.com/github/Shiv1143/Background_Gnereation/blob/master/Different_ways_of_conversion_to_pytorch_JIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Solution 1

In this version we are passing three distinct inputs to the model

In [ ]:
import torch
import torch.nn as nn

class RandomModel(nn.Module):
    def __init__(self):
        super(RandomModel, self).__init__()

        # Define layers for each input
        self.input_ids_fc = nn.Linear(512, 256)
        self.attention_mask_fc = nn.Linear(512, 256)
        self.bbox_fc = nn.Linear(512*4, 256)  # Flatten bbox to 1D

        # Define common layers
        self.common_fc1 = nn.Linear(256*3, 512)
        self.common_fc2 = nn.Linear(512, 128)
        self.output_fc = nn.Linear(128, 1)  # Output layer

        # Define activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, bbox):
        # Pass each input through respective layers
        input_ids_out = self.relu(self.input_ids_fc(input_ids))
        attention_mask_out = self.relu(self.attention_mask_fc(attention_mask))
        bbox_out = self.relu(self.bbox_fc(bbox.view(-1, 512*4)))  # Flatten bbox

        # Concatenate the outputs
        combined = torch.cat((input_ids_out, attention_mask_out, bbox_out), dim=1)

        # Common layers
        common_out = self.relu(self.common_fc1(combined))
        common_out = self.relu(self.common_fc2(common_out))

        # Output layer
        output = self.sigmoid(self.output_fc(common_out))

        return output

# Instantiate the model
model = RandomModel()

# Example input shapes
input_ids = torch.randn(1, 512)
attention_mask = torch.randn(1, 512)
bbox = torch.randn(1, 512, 4)

# Forward pass
output = model(input_ids, attention_mask, bbox)
print("Output shape:", output.shape)


Output shape: torch.Size([1, 1])


In [ ]:
scripted_model = torch.jit.script(model)

In [ ]:
scripted_model.save("random_model.pt")

In [ ]:
# NOTE: delete the previously loaded model, so that it does not fit all the models to GPU

del scripted_model
del model
torch.cuda.synchronize()

In [ ]:
loaded_scripted_model = torch.jit.load("random_model.pt")

# Example input shapes
input_ids = torch.randn(1, 512)
attention_mask = torch.randn(1, 512)
bbox = torch.randn(1, 512, 4)

In [ ]:
output = loaded_scripted_model(input_ids, attention_mask, bbox)
print("Output shape:", output.shape)

Output shape: torch.Size([1, 1])


### Solution 2

In solution 2, we are passing inputs as a dict instead of passing three different inputs

In [ ]:
import torch
import torch.nn as nn

class RandomModel(nn.Module):
    def __init__(self):
        super(RandomModel, self).__init__()

        # Define layers for each input
        self.input_ids_fc = nn.Linear(512, 256)
        self.attention_mask_fc = nn.Linear(512, 256)
        self.bbox_fc = nn.Linear(512*4, 256)  # Flatten bbox to 1D

        # Define common layers
        self.common_fc1 = nn.Linear(256*3, 512)
        self.common_fc2 = nn.Linear(512, 128)
        self.output_fc = nn.Linear(128, 1)  # Output layer

        # Define activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        # Unpack the dictionary
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        bbox = inputs['bbox']

        # Pass each input through respective layers
        input_ids_out = self.relu(self.input_ids_fc(input_ids))
        attention_mask_out = self.relu(self.attention_mask_fc(attention_mask))
        bbox_out = self.relu(self.bbox_fc(bbox.view(-1, 512*4)))  # Flatten bbox

        # Concatenate the outputs
        combined = torch.cat((input_ids_out, attention_mask_out, bbox_out), dim=1)

        # Common layers
        common_out = self.relu(self.common_fc1(combined))
        common_out = self.relu(self.common_fc2(common_out))

        # Output layer
        output = self.sigmoid(self.output_fc(common_out))

        return output

# Instantiate the model
model = RandomModel()

In [ ]:
# Saving to TorchScript

example_input_dict = {'input_ids': torch.randn(1, 512),
                      'attention_mask': torch.randn(1, 512),
                      'bbox': torch.randn(1, 512, 4)}

traced_model = torch.jit.trace(model, example_input_dict)

traced_model.save("random_model_scripted.pt")

In [ ]:
loaded_traced_model = torch.jit.load("random_model_scripted.pt")

# Example input dictionary
input_dict = {'input_ids': torch.randn(1, 512),
              'attention_mask': torch.randn(1, 512),
              'bbox': torch.randn(1, 512, 4)}

# Forward pass
output = loaded_traced_model(input_dict)
print("Output shape:", output.shape)

Output shape: torch.Size([1, 1])


### Solution 3

Input: Dict

Output: Dict

In [ ]:
import torch
import torch.nn as nn

class RandomModel(nn.Module):
    def __init__(self):
        super(RandomModel, self).__init__()

        # Define layers for each input
        self.input_ids_fc = nn.Linear(512, 256)
        self.attention_mask_fc = nn.Linear(512, 256)
        self.bbox_fc = nn.Linear(512*4, 256)  # Flatten bbox to 1D

        # Define common layers
        self.common_fc1 = nn.Linear(256*3, 512)
        self.common_fc2 = nn.Linear(512, 128)

        # Define output layers for each output key
        self.output_fc1 = nn.Linear(128, 1)  # Output layer for 'output1'
        self.output_fc2 = nn.Linear(128, 1)  # Output layer for 'output2'

        # Define activation functions
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, inputs):
        # Unpack the dictionary
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        bbox = inputs['bbox']

        # Pass each input through respective layers
        input_ids_out = self.relu(self.input_ids_fc(input_ids))
        attention_mask_out = self.relu(self.attention_mask_fc(attention_mask))
        bbox_out = self.relu(self.bbox_fc(bbox.view(-1, 512*4)))  # Flatten bbox

        # Concatenate the outputs
        combined = torch.cat((input_ids_out, attention_mask_out, bbox_out), dim=1)

        # Common layers
        common_out = self.relu(self.common_fc1(combined))
        common_out = self.relu(self.common_fc2(common_out))

        # Output layers for each output key
        output1 = self.sigmoid(self.output_fc1(common_out))
        output2 = self.sigmoid(self.output_fc2(common_out))

        # Construct the output dictionary
        output_dict = {'output1': output1, 'output2': output2}

        return output_dict


In [ ]:
model = RandomModel()

In [ ]:
example_input_dict = {'input_ids': torch.randn(1, 512),
                      'attention_mask': torch.randn(1, 512),
                      'bbox': torch.randn(1, 512, 4)}

traced_model = torch.jit.trace(model, example_input_dict, strict=False)

# Save the TorchScript model to file
traced_model.save("random_model_scripted_dict.pt")

# Load the TorchScript model from file
loaded_traced_model = torch.jit.load("random_model_scripted_dict.pt")

In [ ]:
output_dict = loaded_traced_model(example_input_dict)

In [ ]:
output_dict

{'output1': tensor([[0.5068]], grad_fn=<SigmoidBackward0>),
 'output2': tensor([[0.4901]], grad_fn=<SigmoidBackward0>)}